In [1]:
class Operation():
    
    def __init__(self,input_nodes=[]):
        
        self.input_nodes = input_nodes
        self.output_nodes =[]
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_praph.operations.append(self)
    
    def compute(self):
        pass 

In [33]:
class add(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        return x_var + y_var

In [29]:
class multiply(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        return x_var * y_var

In [32]:
class matmul(Operation):
    
    def __init__(self,x,y):
        
        super().__init__([x,y])
    
    def compute(self, x_var, y_var):
        self.inputs = [x_var,y_var]
        
        return x_var.dot(y_var)

In [20]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)

In [21]:
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)

In [22]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self

In [23]:
g = Graph()

In [24]:
g.set_as_default()

In [25]:
A = Variable(10)

In [26]:
b = Variable(1)

In [27]:
x = Placeholder()

In [30]:
y = multiply(A,x)

In [34]:
z = add(y,b)